In [1]:
import pandas as pd

In [2]:
df_weather = pd.read_csv('./data/weather_data_eng.csv')
df_weather.info()

/var/folders/sg/hlv7r8p13tj6dbrhj1phzxlm0000gn/T/ipykernel_20234/1413179625.py:1: DtypeWarning: Columns (4,7,8,9,11,12,16,24,27,29,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_weather = pd.read_csv('./data/weather_data_eng.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2450000 entries, 0 to 2449999
Data columns (total 39 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   id                      object 
 1   type                    object 
 2   longitude               float64
 3   latitude                float64
 4   TEMP_FLAG               object 
 5   HUMIDEX_FLAG            float64
 6   LOCAL_MONTH             int64  
 7   PRECIP_AMOUNT_FLAG      object 
 8   WINDCHILL_FLAG          object 
 9   WIND_SPEED_FLAG         object 
 10  LOCAL_DATE              object 
 11  RELATIVE_HUMIDITY_FLAG  object 
 12  WEATHER_ENG_DESC        object 
 13  LOCAL_HOUR              int64  
 14  RELATIVE_HUMIDITY       float64
 15  LOCAL_DAY               int64  
 16  STATION_PRESSURE_FLAG   object 
 17  WIND_SPEED              float64
 18  VISIBILITY              float64
 19  LOCAL_YEAR              int64  
 20  UTC_DAY                 int64  
 21  TEMP                    float64

In [21]:
WIND_SPEED_THRESHOLD = 50.0
# > 50 km/h is considered Wind storm
SNOW_PRECIP_THRESHOLD = 30.0
# > 30 mm is considered Snow storm
RAIN_PRECIP_THRESHOLD = 20.0
# > 20 mm is considered Rain storm
"""
severe_wind_days = combined_df[combined_df['WIND_SPEED']
                               > WIND_SPEED_THRESHOLD]

winter_storm_days = combined_df[(combined_df['type'] == 'snow') & (
    combined_df['PRECIP_AMOUNT'] >= SNOW_PRECIP_THRESHOLD)]

rainny_storm = combined_df[(combined_df['type'] == 'rain') & (combined_df['PRECIP_AMOUNT'] >= RAIN_PRECIP_THRESHOLD)]
"""

storm_severe = df_weather[df_weather['WIND_SPEED'] > WIND_SPEED_THRESHOLD]
snow_severe = df_weather[(df_weather['WEATHER_ENG_DESC'].str.contains('snow showers')) & (
    df_weather['PRECIP_AMOUNT'] >= SNOW_PRECIP_THRESHOLD)]

In [19]:
storm_severe.head()
storm_severe.to_csv('./storm_severe.csv', index=False)


In [20]:
snow_severe.head()
snow_severe.to_csv('./snow_severe.csv', index=False)

In [25]:
weather_eng_desc_unique = df_weather['WEATHER_ENG_DESC'].unique()
# Print each unique value line by line
for value in weather_eng_desc_unique:
    if isinstance(value, str) and 'Snow' in value:
        print(value)

Snow,Fog
Snow
Snow,Blowing Snow
Snow,Ice Pellets
Snow Showers
Snow Showers,Blowing Snow
Moderate Snow
Ice Pellet Showers,Snow Showers
Moderate Snow Showers
Snow,Ice Pellets,Fog
Rain,Snow
Rain,Snow,Fog
Snow Showers,Fog
Freezing Rain,Snow Pellets
Freezing Drizzle,Snow Pellets
Freezing Drizzle,Snow
Moderate Snow,Freezing Fog
Heavy Snow
Drizzle,Snow,Fog
Heavy Snow Showers
Snow Pellets
Rain Showers,Snow Pellets
Rain Showers,Snow Showers
Ice Pellet Showers,Snow Showers,Fog
Blowing Snow
Snow Grains
Snow Showers,Snow Pellets
Freezing Rain,Snow
Heavy Snow Showers,Blowing Snow
Snow,Snow Grains
Freezing Rain,Snow,Fog
Snow Grains,Fog
Moderate Snow,Blowing Snow
Moderate Snow Showers,Blowing Snow
Freezing Drizzle,Snow,Fog
Snow,Snow Showers
Freezing Rain,Ice Pellets,Blowing Snow
Snow,Ice Pellets,Blowing Snow
Rain Showers,Snow Showers,Fog
Freezing Drizzle,Snow,Blowing Snow
Freezing Rain,Snow,Ice Pellets
Rain,Snow Pellets
Snow Showers,Snow Pellets,Blowing Snow
Freezing Rain,Snow Showers
Drizzle,Snow
Fr